# N starvation 

## Which N sources allow MED4 to grow

check out Escher visualization https://escher.readthedocs.io/en/latest/getting_started.html

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cobra
import numpy as np
import seaborn as sns
import itertools
from matplotlib.colors import LogNorm, Normalize


In [2]:
import subprocess
import time


In [3]:
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import production_envelope


In [4]:
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [5]:
sns.hls_palette

<function seaborn.palettes.hls_palette(n_colors=6, h=0.01, l=0.6, s=0.65, as_cmap=False)>

In [6]:
# values from Ofaim at el.

PARAMETER_VALUES = [#"Name",     "Reaction ID",          "Lower bound", "UpperBound"
                   ["HCO3",      "HCO3EXcar",            -8,            0],
                   #["Nitrogen",  "AmmoniaEX",            -0.56,         0],
                   #["Phosphate", "FAKEOrthophosphateEX", -0.1,          0],
                   ["Light",     "LightEX",              -150,          0]]
CO2MAX = 0.82

# Block fake reactions
FAKE_TRANSPORT = ["AminosugarsOUT", "FAKEAAOUT", "FAKEABPOUT", "FAKEacpTRANS", "FAKEApoacpTRANS", "FAKEThioredoxinTRANS", 
                  "FreefattyacidsOUT", "7NMeth7carbOUT", "ArtificialproteinOUT", "FADOUT", "LipoylproteinTRANS", "MenaquinoneOUT", 
                  "NicotinateOUT", "THFpolyglutOUT", "Thiamin_dpOUT"]


In [7]:
def get_elements(rid):
    r = model.reactions.get_by_id(rid)
    m = r.reactants[0]    
    elements = m.elements.copy()
    
    res = {
        'reaction'   : rid,
        'metabolite' : m.name,
        'mid'        : m.id,
        'N'          : 'N' in elements,
        'C'          : 'C' in elements,
        
    }
    return pd.Series(res)

In [8]:
def alt_get_elements(rid):
    r = alt_model.reactions.get_by_id(rid)
    m = r.reactants[0]    
    elements = m.elements.copy()
    
    res = {
        'reaction'   : rid,
        'metabolite' : m.name,
        'mid'        : m.id,
        'N'          : 'N' in elements,
        'C'          : 'C' in elements,
        
    }
    return pd.Series(res)

# Import model and manipulate based on Ofaim at el

In [9]:
import os
model_dpath = os.path.join('..', 'Model_files')
model_fname = 'iSO595v7_with_formula.xml'
model_fpath = os.path.join(model_dpath, model_fname)


In [10]:
model = cobra.io.read_sbml_model(model_fpath)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


In [11]:
# manipulations copied from Ofaim at el.

remove_blocked = False

# Block H2S
model.reactions.H2SEX.lower_bound = 0

# Block fake transports
for rid in FAKE_TRANSPORT:
    model.reactions.get_by_id(rid).bounds = (0,0)

# Remove blocked reactions
if remove_blocked:
    blocked = cobra.flux_analysis.find_blocked_reactions(model, open_exchanges = True)
    print('blocked', len(blocked), blocked)
    model.remove_reactions([model.reactions.get_by_id(r_id) for r_id in blocked])

# Block maximum CO2 production
model.reactions.CO2EX.bounds = (0, CO2MAX)

for i, row in enumerate(PARAMETER_VALUES):
    # Row: Name, Reaction ID, lower bound, upper bound
    key = row[0]
    reaction_id = row[1]
    lower_bound = row[2]
    upper_bound = row[3]
    r = model.reactions.get_by_id(reaction_id)
    # Fix flux
    r.bounds = (lower_bound, upper_bound)

In [12]:
exchange_ids = [r.id for r in model.exchanges] # + ["R00024"]

In [13]:
print('reactions', len(model.reactions))
print('metabolites', len(model.metabolites))
print('genes', len(model.genes))

reactions 994
metabolites 802
genes 595


In [14]:
model

Name,COBRAModel
Memory address,0x07f8456ea8c70
Number of metabolites,802
Number of reactions,994
Number of groups,55
Objective expression,1.0*BIOMASS - 1.0*BIOMASS_reverse_69053
Compartments,"Cytoplasm, Extracellular, unknownCompartment2, Periplasm, unknownCompartment5"


In [15]:
model.metabolites.get_by_id('Carbamate[c]')

Metabolite identifier,Carbamate[c]
Name,Carbamate
Memory address,0x7f8456e6c040
Formula,CH3NO2
Compartment,c
In 4 reaction(s),"R03546, R01395, R10949, R07316"


In [16]:
cobra.Configuration()

Attribute,Description,Value
solver,Mathematical optimization solver,gurobi
tolerance,"General solver tolerance (feasibility, integrality, etc.)",1e-07
lower_bound,Default reaction lower bound,-1000.0
upper_bound,Default reaction upper bound,1000.0
processes,Number of parallel processes,11
cache_directory,Path for the model cache,/home/oweissberg/.cache/cobrapy
max_cache_size,Maximum cache size in bytes,104857600
cache_expiration,Model cache expiration time in seconds (if any),None


In [17]:
model.reactions.get_by_id('BIOMASS')

Reaction identifier,BIOMASS
Name,Biomass formation
Memory address,0x7f8456c7b760
Stoichiometry,104.0 ATP[c] + 0.0324116276714271 BioPool[c] + 0.0268408791654006 Bmineral[c] + 0.0559910868023904 Cell_wall[c] + 0.00111921401802897 Free_nucleic_acids[c] + 104.0 H2O[c] + 0.649245416793275... 104.0 ATP + 0.0324116276714271 BioPool + 0.0268408791654006 Trace metals and minerals + 0.0559910868023904 Cell wall Biomass precursor + 0.00111921401802897 Free nucleic acids + 104.0 H2O +...
GPR,
Lower bound,0.0
Upper bound,1000.0


# Get a map of med4 metabolites to BiGG ids

In [18]:
bigg_df =  pd.read_excel('MED4_to_Bigg.xlsx',)

# Compare exchange reactions/metabolites of the two models

In [19]:
exchange_ids = [r.id for r in model.exchanges] 
pro_med_df = pd.DataFrame({'mediumExchange' : exchange_ids})
pro_med_df = pro_med_df.mediumExchange.apply(get_elements)


In [20]:
#pro_df = pd.read_excel('PRO_N_sources.xlsx', sheet_name='secretion')
alt_dpath = '../../HOT1A3-model/N_Starvation/'
#alt_med_df = pd.read_excel(os.path.join(alt_dpath, 'HOT1A3_medium_NCBI.xlsx'))
alt_model_dpath = os.path.join('../../HOT1A3-model', 'Models')
alt_model_fname = 'HOT1A3_ncbi.xml'
alt_model_fpath = os.path.join(alt_model_dpath, alt_model_fname)


In [21]:
alt_model = cobra.io.read_sbml_model(alt_model_fpath)
alt_exchange_ids = [r.id for r in alt_model.exchanges] 
alt_med_df = pd.DataFrame({'mediumExchange' : alt_exchange_ids})
alt_med_df = alt_med_df.mediumExchange.apply(alt_get_elements)


In [22]:
pro_med_df = pd.merge(pro_med_df, bigg_df[['mid', 'BiggID']].drop_duplicates(), on='mid', how='left')

In [23]:
pro_med_df.head()

,reaction,metabolite,mid,N,C,BiggID
0,AminosugarsEX,Amino sugars pool,Amino_sugars[e],False,False,NaN
1,FAKEAAEX,amino_acid_pool_e,amino_acid_pool[e],False,False,NaN
2,FAKEABPEX,Adenosine 3_5-bisphosphate,Adenosine_3_5_bisphosphate[e],True,True,pap
3,FAKEacpEX,Acyl-carrier protein,Acyl_carrier_protein[e],False,False,ACP
4,FAKEApoacpEX,Apo-(acyl-carrier-protein),Apo_acyl_carrier_protein[e],False,False,NaN


In [24]:
alt_med_df.head()

,reaction,metabolite,mid,N,C
0,EX_14glucan_e,"1,4-alpha-D-glucan",14glucan_e,False,True
1,EX_25dkglcn_e,"2,5-diketo-D-gluconate",25dkglcn_e,False,True
2,EX_2ameph_e,2-Aminoethylphosphonate,2ameph_e,True,True
3,EX_2m35mdntha_e,N 2 methyl 3 5 dinitrophenyl 4 methyl 3 5 di...,2m35mdntha_e,True,False
4,EX_2pglyc_e,2-Phosphoglycolate,2pglyc_e,False,True


In [25]:
pro_med_df.shape

(102, 6)

In [26]:
alt_med_df.shape

(257, 5)

In [27]:
alt_med_df.columns, pro_med_df.columns

(Index(['reaction', 'metabolite', 'mid', 'N', 'C'], dtype='object'),
 Index(['reaction', 'metabolite', 'mid', 'N', 'C', 'BiggID'], dtype='object'))

In [28]:
alt_med_df['BiggID'] = alt_med_df.mid.str.rpartition('_',)[0]

In [29]:
exchange_df = pd.merge(pro_med_df, alt_med_df, left_on='BiggID', right_on='BiggID', how='outer', suffixes=['_pro', '_alt'])

In [30]:
exchange_df.BiggID.isna().value_counts()

False    293
True      11
Name: BiggID, dtype: int64

In [31]:
exchange_df.loc[exchange_df.BiggID.str.contains('ala__L', case=False, na=False)]

,reaction_pro,metabolite_pro,mid_pro,N_pro,C_pro,BiggID,reaction_alt,metabolite_alt,mid_alt,N_alt,C_alt
55,L_AlanineEX,L-Alanine,L_Alanine[e],True,True,ala__L,EX_ala__L_e,L-Alanine,ala__L_e,True,True
240,NaN,NaN,NaN,NaN,NaN,met_L_ala__L,EX_met_L_ala__L_e,Met L ala L C8H16N2O3S,met_L_ala__L_e,True,True


In [32]:
exchange_df.to_excel('exchange.xlsx')

In [33]:
exchange_df['map_status'] = 'mapped'
exchange_df.loc[exchange_df.metabolite_alt.isna(), 'map_status'] = 'PRO only'
exchange_df.loc[exchange_df.metabolite_pro.isna(), 'map_status'] = 'ALT only'
exchange_df.map_status.value_counts()

ALT only    202
mapped       55
PRO only     47
Name: map_status, dtype: int64

In [34]:
exchange_df.isna().sum()

reaction_pro      202
metabolite_pro    202
mid_pro           202
N_pro             202
C_pro             202
BiggID             11
reaction_alt       47
metabolite_alt     47
mid_alt            47
N_alt              47
C_alt              47
map_status          0
dtype: int64

# Look for potential cross feeding

In [35]:
pro_df = pd.read_excel('data/PRO_N_sources.xlsx', sheet_name='secretion')
alt_dpath = '../../HOT1A3-model/N_Starvation/'
alt_df = pd.read_excel(os.path.join(alt_dpath, 'ALT_N_sources.xlsx'), sheet_name='secretion')
alt2_df = pd.read_excel(os.path.join(alt_dpath, 'ALT_N_sources.xlsx'), sheet_name='secretion2')


In [36]:
alt_df.head()

,Unnamed: 0,secretion,minimum,maximum,uptake,metabolite_uptake,mid_uptake,N_uptake,C_uptake,metabolite_secretion,mid_secretion,N_secretion,C_secretion
0,0,sink_2ohph_c,0.0,66.658441,EX_cytd_e,Cytidine,cytd_e,True,True,2-Octaprenyl-6-hydroxyphenol,2ohph_c,False,True
1,1,sink_4crsol_c,0.0,279.476874,EX_cytd_e,Cytidine,cytd_e,True,True,P-Cresol,4crsol_c,False,True
2,3,sink_hemeO_c,0.0,65.829612,EX_cytd_e,Cytidine,cytd_e,True,True,Heme O C49H56FeN4O5,hemeO_c,True,True
3,4,sink_mobd_c,0.0,1000.000000,EX_cytd_e,Cytidine,cytd_e,True,True,Molybdate,mobd_c,False,False
4,5,sink_sheme_c,0.0,95.624557,EX_cytd_e,Cytidine,cytd_e,True,True,Siroheme C42H36FeN4O16,sheme_c,True,True


In [37]:
alt2_df.loc[alt2_df.N_uptake, 'metabolite_uptake']

253      Ammonium
254      Ammonium
255      Ammonium
256      Ammonium
257      Ammonium
           ...   
20191     Cyanate
20192     Cyanate
20193     Cyanate
20194     Cyanate
20195     Cyanate
Name: metabolite_uptake, Length: 5880, dtype: object

In [38]:
alt2_df.head()

,Unnamed: 0,secretion,minimum,maximum,uptake,uptake2,metabolite_uptake,mid_uptake,N_uptake,C_uptake,metabolite_uptake2,mid_uptake2,N_uptake2,C_uptake2,metabolite_secretion,mid_secretion,N_secretion,C_secretion
0,0,sink_2ohph_c,0.0,68.865821,EX_14glucan_e,EX_nh4_e,"1,4-alpha-D-glucan",14glucan_e,False,True,Ammonium,nh4_e,True,False,2-Octaprenyl-6-hydroxyphenol,2ohph_c,False,True
1,1,sink_4crsol_c,0.0,308.174662,EX_14glucan_e,EX_nh4_e,"1,4-alpha-D-glucan",14glucan_e,False,True,Ammonium,nh4_e,True,False,P-Cresol,4crsol_c,False,True
2,3,sink_hemeO_c,0.0,59.261590,EX_14glucan_e,EX_nh4_e,"1,4-alpha-D-glucan",14glucan_e,False,True,Ammonium,nh4_e,True,False,Heme O C49H56FeN4O5,hemeO_c,True,True
3,4,sink_mobd_c,0.0,1000.000000,EX_14glucan_e,EX_nh4_e,"1,4-alpha-D-glucan",14glucan_e,False,True,Ammonium,nh4_e,True,False,Molybdate,mobd_c,False,False
4,5,sink_sheme_c,0.0,77.873881,EX_14glucan_e,EX_nh4_e,"1,4-alpha-D-glucan",14glucan_e,False,True,Ammonium,nh4_e,True,False,Siroheme C42H36FeN4O16,sheme_c,True,True


In [39]:
pro_df.columns

Index(['Unnamed: 0', 'secretion', 'minimum', 'maximum', 'uptake',
       'metabolite_uptake', 'mid_uptake', 'N_uptake', 'C_uptake',
       'metabolite_secretion', 'mid_secretion', 'N_secretion', 'C_secretion'],
      dtype='object')

In [43]:
#pro_df = 
pro_df = pd.merge(pro_df, bigg_df[['mid', 'BiggID', 'name_ref']].drop_duplicates(), 
         left_on='mid_uptake', right_on='mid', how='left', suffixes=['', '_uptake'] )
pro_df = pd.merge(pro_df, bigg_df[['mid', 'BiggID', 'name_ref']].drop_duplicates(), 
         left_on='mid_secretion', right_on='mid', how='left', suffixes=['', '_secretion'] )


In [45]:
pro_df.head()

,Unnamed: 0,secretion,minimum,maximum,uptake,metabolite_uptake,mid_uptake,N_uptake,C_uptake,metabolite_secretion,...,N_secretion,C_secretion,metabolite1_uptake,metabolite1_secretion,mid,BiggID,name_ref,mid_secretion,BiggID_secretion,name_ref_secretion
0,0,B12EX,0.000000,7.941910,GuanosineEX,Guanosine[e],Guanosine[e],True,True,Cobamide coenzyme,...,True,True,gsn,adocbl,Guanosine[e],gsn,Guanosine,Cobamide_coenzyme[e],adocbl,Calomide
1,1,BiotinEX,0.000000,1.985477,GuanosineEX,Guanosine[e],Guanosine[e],True,True,Biotin,...,True,True,gsn,btn,Guanosine[e],gsn,Guanosine,Biotin[e],btn,BIOT
2,2,MethanolEX,0.000042,1.985519,GuanosineEX,Guanosine[e],Guanosine[e],True,True,Methanol,...,False,True,gsn,meoh,Guanosine[e],gsn,Guanosine,Methanol[e],meoh,Methanol
3,3,SAdenosyl4methylthio2oxobutanoateEX,0.000042,1.985519,GuanosineEX,Guanosine[e],Guanosine[e],True,True,S-Adenosyl-4-methylthio-2-oxobutanoate,...,True,True,gsn,amob,Guanosine[e],gsn,Guanosine,S_Adenosyl_4_methylthio_2_oxobutanoate[e],amob,S-Adenosyl-4-methylthio-2-oxobutanoate
4,4,SMethyl5thioDriboseEX,0.000000,15.884229,GuanosineEX,Guanosine[e],Guanosine[e],True,True,S-Methyl-5-thio-D-ribose,...,False,True,gsn,5mtr,Guanosine[e],gsn,Guanosine,S_Methyl_5_thio_D_ribose[e],5mtr,5-Methylthio-D-ribose


In [46]:
alt_uptake = (set(alt_df.metabolite_uptake.unique()) | set(alt2_df.metabolite_uptake.unique()) | 
              set(alt2_df.metabolite_uptake2.unique()))
alt1_uptake = set(alt_df.metabolite_uptake.unique()) 
alt2_uptake = set(alt2_df.metabolite_uptake.unique()) | set(alt2_df.metabolite_uptake2.unique())
pro_uptake = set(pro_df.name_ref.unique())
alt_secretion = set(alt_df.metabolite_secretion.unique()) | set(alt2_df.metabolite_secretion.unique())
pro_secretion = set(pro_df.name_ref_secretion.unique())


In [47]:
alt2_N_metabolites = (
    set(alt2_df.loc[alt2_df.N_uptake, 'metabolite_uptake'].unique()) | 
    set(alt2_df.loc[alt2_df.N_uptake2, 'metabolite_uptake2'].unique()) 
)
alt2_C_metabolites = (
    set(alt2_df.loc[alt2_df.C_uptake, 'metabolite_uptake'].unique()) | 
    set(alt2_df.loc[alt2_df.C_uptake2, 'metabolite_uptake2'].unique()) 
)



In [48]:
print(
    '\nalt2_N_metabolites', len(alt2_N_metabolites), 
    '\nalt2_C_metabolites', len(alt2_C_metabolites), 
    '\nalt2_C+N_metabolites', len(alt2_C_metabolites & alt2_N_metabolites), 
    '\nalt2_none_metabolites', len(alt2_uptake - (alt2_C_metabolites | alt2_N_metabolites)), 
    
)


alt2_N_metabolites 7 
alt2_C_metabolites 80 
alt2_C+N_metabolites 3 
alt2_none_metabolites 1


In [49]:
', '.join(sorted(alt2_N_metabolites - alt2_C_metabolites))

'2 4 6 Trinitrotoluene, Ammonium, Nitrate, Nitrite'

In [50]:
', '.join(sorted(alt2_C_metabolites - alt2_N_metabolites))

' R  Acetoin C4H8O2, (R)-3-Hydroxybutanoate, 1,4-alpha-D-glucan, 1-Propanol, 10 Phenyldecanoic acid, 2,5-diketo-D-gluconate, 2-Phosphoglycolate, 3-Methylbutanoic acid, 4-Hydroxyphenylacetate, 5 phenylvaleric acid, 5-Dehydro-D-gluconate, 6 Phenylhexanoic acid, 6 acetylthiohexanoic acid, 7 Phenylheptanoic acid, 8 Phenyloctanoic acid, 9 Phenylnonanoic acid, Acetaldehyde, Acetate, Acetoacetate, Beta D-Galactose, Butyrate (n-C4:0), Carbon monoxide, Cellobiose, Citrate, D-Fructose, D-Galactose, D-Galacturonate, D-Glucose, D-Glucuronate, D-Lactate, D-Mannitol, D-Mannose, D-Ribose, D-Sorbitol, Decanoate (n-C10:0), Deoxyribose C5H10O4, Dodecanoate (n-C12:0), Ethanol, Fe(III)dicitrate, Formate, Fumarate, Galactitol, Glycerol, Glycerol 2-phosphate, Glycerophosphoglycerol, Glycol, Heptanoate, Hexadecanoate (n-C16:0), Hexadecenoate (n-C16:1), Hexanoate (n-C6:0), Isobutyrate, L-Ascorbate, L-Fucose, L-Idonate, L-Lactate, L-Malate, Lactose C12H22O11, Linoleic acid (all cis C18:2) n-6, Maltopentaose, M

In [51]:
', '.join(sorted(alt2_C_metabolites & alt2_N_metabolites))

'Cyanate, Phenethylamine, Urea CH4N2O'

In [52]:
', '.join(sorted(alt2_uptake - (alt2_C_metabolites | alt2_N_metabolites)))

'Hydrogen sulfide'

In [53]:
print(
    '\nalt->pro', len(pro_uptake & alt_secretion), 
    '\nalt_secretion', len(alt_secretion), 
    '\npro_uptake', len(pro_uptake), 
    '\npro->alt', len(alt_uptake & pro_secretion), 
    '\npro_secretion', len(pro_secretion),
    '\nalt_uptake', len(alt_uptake), 
    '\nalt1_uptake', len(alt1_uptake), 
    '\nalt2_uptake', len(alt2_uptake), 
)


alt->pro 4 
alt_secretion 53 
pro_uptake 17 
pro->alt 25 
pro_secretion 58 
alt_uptake 152 
alt1_uptake 67 
alt2_uptake 85


In [55]:
', '.join(sorted(alt_uptake & pro_secretion))

'Acetate, Adenine, Citrate, Ethanol, Formate, Fumarate, Glycine, Guanine, L-Alanine, L-Arginine, L-Asparagine, L-Aspartate, L-Glutamate, L-Glutamine, L-Histidine, L-Isoleucine, L-Lysine, L-Malate, L-Proline, L-Serine, Methanol, Putrescine, Pyruvate, Succinate, Xanthosine'

In [56]:
', '.join(sorted(alt1_uptake))

'2-Aminoethylphosphonate, Adenine, Adenosine, Ala L asp L C7H11N2O5, Ala L glu L C8H13N2O5, Ala-Gln, Ala-His, Ala-L-Thr-L, Allantoin, Choline C5H14NO, Choline sulfate, Cys Gly C5H10N2O3S, Cytidine, Cytosine, D-Alanine, Gly asn L C6H11N3O4, Gly asp L C6H9N2O5, Gly glu L C7H11N2O5, Gly pro L C7H12N2O3, Gly-Cys, Gly-Gln, Gly-Phe, Gly-Tyr, Glycerophosphoserine, Glycine, Glycine betaine, Glycine-glycine-glutamine tripeptide, Glycylglycine C4H8N2O3, Glycylphenylalanine, Guanine, Hypoxanthine, Inosine, L alaninylhistidine, L alaninylthreonine, L alaninyltryptophan, L glycinylglutamate, L glycinylglutamine, L glycinylserine, L histidinylglycine, L histidinylhistidine, L-Alanine, L-Arginine, L-Asparagine, L-Aspartate, L-Carnitine, L-Cysteine, L-Glutamate, L-Glutamine, L-Histidine, L-Isoleucine, L-Lysine, L-Proline, L-Prolinylglycine, L-Serine, L-alanine-L-glutamate, L-alanylglycine, Lysine-glutamine-glycine tripeptide, NMN C11H14N2O8P, Ornithine, Putrescine, Reduced glutathione, Serine-glutamin

In [57]:
', '.join(sorted(alt2_uptake))

' R  Acetoin C4H8O2, (R)-3-Hydroxybutanoate, 1,4-alpha-D-glucan, 1-Propanol, 10 Phenyldecanoic acid, 2 4 6 Trinitrotoluene, 2,5-diketo-D-gluconate, 2-Phosphoglycolate, 3-Methylbutanoic acid, 4-Hydroxyphenylacetate, 5 phenylvaleric acid, 5-Dehydro-D-gluconate, 6 Phenylhexanoic acid, 6 acetylthiohexanoic acid, 7 Phenylheptanoic acid, 8 Phenyloctanoic acid, 9 Phenylnonanoic acid, Acetaldehyde, Acetate, Acetoacetate, Ammonium, Beta D-Galactose, Butyrate (n-C4:0), Carbon monoxide, Cellobiose, Citrate, Cyanate, D-Fructose, D-Galactose, D-Galacturonate, D-Glucose, D-Glucuronate, D-Lactate, D-Mannitol, D-Mannose, D-Ribose, D-Sorbitol, Decanoate (n-C10:0), Deoxyribose C5H10O4, Dodecanoate (n-C12:0), Ethanol, Fe(III)dicitrate, Formate, Fumarate, Galactitol, Glycerol, Glycerol 2-phosphate, Glycerophosphoglycerol, Glycol, Heptanoate, Hexadecanoate (n-C16:0), Hexadecenoate (n-C16:1), Hexanoate (n-C6:0), Hydrogen sulfide, Isobutyrate, L-Ascorbate, L-Fucose, L-Idonate, L-Lactate, L-Malate, Lactose C1

In [58]:
', '.join(sorted(pro_secretion))

'4MOP, 5-Methylthio-D-ribose, 5-Methylthioadenosine, ABEE, Acetate, Adenine, BIOT, Calomide, Citrate, Dethiobiotin, Ethanol, Formate, Fumarate, GSH, Glycerol-3-phosphate, Glycine, Glycogen, Glycolate, Guanine, Guanosine, H2O2, H2S, H2S2O3, L-Alanine, L-Arginine, L-Asparagine, L-Aspartate, L-Cystine, L-Glutamate, L-Glutamine, L-Histidine, L-Isoleucine, L-Leucine, L-Lysine, L-Malate, L-Methionine, L-Phenylalanine, L-Proline, L-Serine, L-Threonine, L-Tryptophan, L-Tyrosine, L-Valine, Lipid A disaccharide, Methanol, NH3, NH4+, PAN, Putrescine, Pyruvate, S-Adenosyl-4-methylthio-2-oxobutanoate, S2-, Spermidine, Spermine, Succinate, Thymidine, Urea, Xanthosine'

In [59]:
', '.join(sorted(pro_uptake & alt_secretion))

'Guanine, L-Aspartate, L-Serine, L-Valine'

In [60]:
', '.join(sorted(pro_uptake))

'Cyanate, GSH, Glycine, Guanine, Guanosine, L-Alanine, L-Arginine, L-Aspartate, L-Cystine, L-Glutamate, L-Glutamine, L-Leucine, L-Serine, L-Valine, NH3, NH4+, Urea'

In [61]:
', '.join(sorted(alt_secretion))

"(2R,4S)-2-methyl-2,3,3,4-tetrahydroxytetrahydrofuran, 2-Octaprenyl-6-hydroxyphenol, 3 Hydroxy 4 acetylthiobutanoic acid, 4-Hydroxybenzoate, 5'-deoxyribose, 5-Methylthio-5-deoxy-D-ribulose 1-phosphate, Acetaldehyde, Ammonium, Benzoate, Cellobiose, Choline C5H14NO, D-Galactose, D-Glucose, D-Ribose, Ethanol, Ethanolamine, Fe2+ mitochondria, Formaldehyde, Formate, Glycerol, Glycerol 3-phosphate, Glycerophosphoglycerol, Glycine betaine, Guanine, Heme O C49H56FeN4O5, Hydrogen sulfide, Indole, Indole 3 acetate C10H8NO2, L-Aspartate, L-Homoserine, L-Lysine, L-Serine, L-Valine, Methanol, Molybdate, N  2 methyl 3 5 dinitrophenyl  4 methyl 3 5 dinitroaniline, N N bis 3 5 dinitrotolyl  amine, Nitrite, Ornithine, Oxalate, P-Cresol, Phenylacetaldehyde, Phosphate, Protoheme C34H30FeN4O4, Selenite, Siroheme C42H36FeN4O16, Sn-Glycero-3-phosphoethanolamine, Spermidine, Sulfur, Thymine C5H6N2O2, Trans-Cinnamate, Uracil, Urea CH4N2O"

In [62]:
alt_uptake | pro_uptake | alt_secretion | pro_secretion

{' R  Acetoin C4H8O2',
 '(2R,4S)-2-methyl-2,3,3,4-tetrahydroxytetrahydrofuran',
 '(R)-3-Hydroxybutanoate',
 '1,4-alpha-D-glucan',
 '1-Propanol',
 '10 Phenyldecanoic acid',
 '2 4 6 Trinitrotoluene',
 '2,5-diketo-D-gluconate',
 '2-Aminoethylphosphonate',
 '2-Octaprenyl-6-hydroxyphenol',
 '2-Phosphoglycolate',
 '3 Hydroxy 4 acetylthiobutanoic acid',
 '3-Methylbutanoic acid',
 '4-Hydroxybenzoate',
 '4-Hydroxyphenylacetate',
 '4MOP',
 '5 phenylvaleric acid',
 "5'-deoxyribose",
 '5-Dehydro-D-gluconate',
 '5-Methylthio-5-deoxy-D-ribulose 1-phosphate',
 '5-Methylthio-D-ribose',
 '5-Methylthioadenosine',
 '6 Phenylhexanoic acid',
 '6 acetylthiohexanoic acid',
 '7 Phenylheptanoic acid',
 '8 Phenyloctanoic acid',
 '9 Phenylnonanoic acid',
 'ABEE',
 'Acetaldehyde',
 'Acetate',
 'Acetoacetate',
 'Adenine',
 'Adenosine',
 'Ala L asp L C7H11N2O5',
 'Ala L glu L C8H13N2O5',
 'Ala-Gln',
 'Ala-His',
 'Ala-L-Thr-L',
 'Allantoin',
 'Ammonium',
 'BIOT',
 'Benzoate',
 'Beta D-Galactose',
 'Butyrate (n-C4:0)

In [63]:
pro_df.loc[pro_df.isna().sum(axis=1) > 0]

,Unnamed: 0,secretion,minimum,maximum,uptake,metabolite_uptake,mid_uptake,N_uptake,C_uptake,metabolite_secretion,...,N_secretion,C_secretion,metabolite1_uptake,metabolite1_secretion,mid,BiggID,name_ref,mid_secretion,BiggID_secretion,name_ref_secretion


In [64]:
alt_df.loc[alt_df.isna().sum(axis=1) > 0]

,Unnamed: 0,secretion,minimum,maximum,uptake,metabolite_uptake,mid_uptake,N_uptake,C_uptake,metabolite_secretion,mid_secretion,N_secretion,C_secretion
